In [1]:
from scraper import get_current_events, get_description_and_more_data, get_headers_and_cookies
from classes import Event, DB, Filter

40


In [2]:
events = get_current_events()
description_missing = []
events_new = {}
for event_id, event_data in events.items():
    new_data = get_description_and_more_data(event_data["title"])
    events[event_id].update(new_data)

    if len(events[event_id]["description"]) < 10:
        title = event_data["title"].replace(" ", "-").lower().replace("-–-", "-").replace("---", "-")
        description_missing.append(f"https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/{title}/")
    else:
        events_new[event_id] = events[event_id]

print(len(events_new), len(events))

https://www.planetarium-hamburg.de/files/usr/module/produktionen/planeten-expedition-durchs-sonnensystem/Thumbnail_PlanetenExpeditionDurchsSonnensystem.jpg
https://www.planetarium-hamburg.de/assets/images/9/Thumbnail_WirSindSterne-8e8a2fe4.jpg
https://www.planetarium-hamburg.de/files/default/img/hamburg--logo.png
https://www.planetarium-hamburg.de/assets/images/5/Thumbnail_ImNachtflugDurchDieGalaxis-86e5fbb2.jpg
https://www.planetarium-hamburg.de/files/default/img/hamburg--logo.png
https://www.planetarium-hamburg.de/files/default/img/hamburg--logo.png
https://www.planetarium-hamburg.de/assets/images/b/Thumbnail_Sterntaler-ef92caa8.jpg
https://www.planetarium-hamburg.de/files/default/img/hamburg--logo.png
https://www.planetarium-hamburg.de/files/usr/module/produktionen/die-zukunft-des-universums/Thumbnail_DieZukunftDesUniversums.jpg
https://www.planetarium-hamburg.de/files/default/img/hamburg--logo.png
https://www.planetarium-hamburg.de/assets/images/6/thumbnail-aurora-trailer-7a32da8c.

In [ ]:
print(*description_missing, sep="\n")

https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/polaris-und-das-rätsel-der-polarnacht/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/explore-3d/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/premiere:-die-zukunft-des-universums/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/die-drei-???-das-dorf-der-teufel/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/der-kleine-häwelmann/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/100-jahre-planetarien-special/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/meine-heimat-unser-blauer-planet/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/kleiner-stern-im-großen-all/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/die-drei-???-das-dorf-der-teufel/
https://www.planetarium-hamburg.de/de/veranstaltungen-tickets/details/explore-3d/
https://www.pla